<a href="https://colab.research.google.com/github/pavankumarbalijepalli/pr-phi2-vs-defog/blob/main/inf_phi2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!pip install -q -U torch=='2.0.0' peft=='0.7.1' transformers einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 2.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu121 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.
torchvision 0.16.0+cu121 requires torch==2.1.0, but you have torch 2.0.0 which is incompatible.


In [23]:
!pip install ctransformers>=0.2.24

In [5]:
# Data Exploration Imports
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import transformers
from peft import LoraConfig, PeftConfig
from transformers import (AutoModelForCausalLM,
                          AutoTokenizer,
                          BitsAndBytesConfig,
                          TrainingArguments,
                          pipeline,
                          logging)
from sklearn.metrics import (accuracy_score,
                             classification_report,
                             confusion_matrix)

In [6]:
!nvidia-smi

Wed Feb 21 12:57:20 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [14]:
test_df = pd.read_csv('test.csv')
test_df.head()

,Unnamed: 0,question,context,answer,table_count,sub_query_count,joins_count,where_count,group_by_count,columns_count,complexity,difficulty
0,30251,who directed with original air date being nove...,CREATE TABLE table_14637853_3 (directed_by VAR...,SELECT directed_by FROM table_14637853_3 WHERE...,1,1,0,1,0,2,5,easy
1,37181,List episode directed and produced by Karen Se...,CREATE TABLE table_24725951_1 (episode VARCHAR...,SELECT episode FROM table_24725951_1 WHERE dir...,1,1,0,1,0,2,5,easy
2,76939,Name the 2012 when 2011 is qf,CREATE TABLE table_name_97 (Id VARCHAR),"SELECT 2012 FROM table_name_97 WHERE 2011 = ""qf""",1,1,0,1,0,1,4,easy
3,71941,Who was the best blocker for the year that has...,CREATE TABLE table_name_94 (best_blocker VARCH...,SELECT best_blocker FROM table_name_94 WHERE l...,1,1,0,1,0,3,6,medium
4,29748,What is GTO Winning Team Mike Keyser's RND num...,"CREATE TABLE table_13657749_2 (rnd VARCHAR, gt...",SELECT rnd FROM table_13657749_2 WHERE gto_win...,1,1,0,1,0,2,5,easy


In [8]:
def generate_test_prompt(data_point):
    return f"""### Task
Generate a SQL query to answer the following question:
`{data_point['question']}`

### Database Schema
The query will run on a database with the following schema:
{data_point['context']}

### Answer
Given the database schema, here is the SQL query that answers `{data_point['question']}`:
```sql""".strip()

In [26]:
from llama_cpp import Llama

llm = Llama(
      model_path="./models/7B/llama-model.gguf",
)
output = llm()

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

RuntimeError: Failed to create LLM 'llama' from '/root/.cache/huggingface/hub/models--pavankumarbalijepalli--phi2-nl2sql-lora-merged-gguf/blobs/6c02ba08dd1c527668b5453cc06192a56ddbdcf1b731f74545437bfdeca81141'.

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

phi_tokenizer = AutoTokenizer.from_pretrained("pavankumarbalijepalli/phi2-nl2sql-lora-merged", trust_remote_code=True)
phi_model = AutoModelForCausalLM.from_pretrained("pavankumarbalijepalli/phi2-nl2sql-lora-merged", trust_remote_code=True)

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/35.7k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [20]:
from datetime import datetime

out = {"prompt": [], "pred": [], "actu": [], "inf_time": [], "temperature": [], "difficulty": []}

def predict(df, model, tokenizer, out):
    for point in tqdm(df.iloc, desc="No. of rows", total=df.shape[0]):
      start = datetime.now()
      # out = {"prompt": '', "pred": '', "actu": point['answer'], "inf_time": '', "temperature": '0.2', "difficulty": point['difficulty']}
      prompt = generate_test_prompt(point)
      out['prompt'].append(prompt)
      out['actu'].append(point['answer'])
      pipe = pipeline(task="text-generation",
                      model=model,
                      tokenizer=tokenizer,
                      max_new_tokens = 50,
                      temperature = 0.2,
                      )
      result = pipe(prompt, pad_token_id=pipe.tokenizer.eos_token_id)
      answer = result[0]['generated_text']
      end = datetime.now()
      out['inf_time'].append((end - start).total_seconds())
      out['pred'].append(answer.split('```sql')[-1].split('```')[0].strip())
      out['temperature'].append(0.2)
      out['difficulty'].append(point['difficulty'])
    return out

In [22]:
phi_model.eval()

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (rotary_emb): PhiRotaryEmbedding()
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (final_layernorm): LayerNorm((2560,),

In [21]:
out = predict(test_df, phi_model, phi_tokenizer, out)

No. of rows:   0%|          | 2/7858 [00:59<64:37:29, 29.61s/it]


KeyboardInterrupt: 

In [ ]:
out.to_csv('predictions.csv')

from google.colab import files
files.download('/content/predictions.csv')